In [2]:
import json 
import re
import os 
import pandas as pd
import glob
from tqdm import tqdm
from tqdm.auto import tqdm


dir_path = "/home/tan/Downloads/[Daesu] BB_frame_retouching_30k(AD)_result_230602_54889910/batch_20"

json_paths  = glob.glob(os.path.join(dir_path, '**/*.json'), recursive=True)
print(json_paths)
total_json = []
merged_data = []

for json_path in tqdm(json_paths):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        data['sf_filename'] = os.path.basename(json_path)
        merged_data.append(data)

for json_file in tqdm(merged_data):
    merged_meta = {'batch': re.compile(r'batch_\d+').search(json_file['parent_path']).group(),
                        'filename': json_file['sf_filename'],
                         'parent_path': json_file['parent_path'],
                         'unix_timestamp' : json_file['unix_timestamp'],
                         'file_format' : json_file['file_format'],
                         'capacity' : json_file['capacity'],
                         'vehicle_id' : json_file['vehicle_id'],
                         'region_name' : json_file['region_name'],
                         'location' : json_file['location'],
                         'framerate' : json_file['framerate'],
                         'size' : json_file['size'],
                         'data_purpose': json_file['data_purpose'],
                         'weather':json_file['weather'] ,
                         'time' : json_file['time'] ,
                         'driving_scenario' : json_file['driving_scenario'],
                         'ego_long_vel_level' : json_file['ego_long_vel_level'] ,
                         'season' : json_file['season'],
                         #"cloudiness" : json_file['cloudiness'],
                         "road_feature" : json_file['road_feature'],
                         "location_feature": json_file['location_feature'],
                         "road_type": json_file['road_type'],
                         "sensor_status" : json_file['sensor_status'],
                         "illumination_status" : json_file['illumination_status'],
                         "road_status" : json_file['road_status'],
                         #"road_surface_type": json_file['road_surface_type'],
                         #"road_surface_feature": json_file['road_surface_feature'],
                         #"road_surface_condition": json_file['road_surface_condition'],
                         #"road_surface_contamination_level": json_file['road_surface_contamination_level'],
                         #"shadow_in_lane": json_file['shadow_in_lane'],
                         #"damaged_lane": json_file['damaged_lane'],
                         #"darkness_level": json_file['darkness_level'],
                         #"backlight_level": json_file['backlight_level'],
                         #"backlight_source": json_file['backlight_source'],
                         #"camera_status": json_file['camera_status'],
                         #"contamination_level": json_file['contamination_level'],
                         'crowd_level' : json_file['crowd_level'],
                         "scene_att" : json_file["scene_att"],
                         'sensor_name' : json_file['sensor_name'],
                         'sensor_hfov' : json_file['sensor_hfov'],
                         'sensor_vfov' : json_file['sensor_vfov'],
                         'gps_mode': json_file['gps_mode'],
                         'gps_latitude': json_file['gps_latitude'],
                         'gps_longitude': json_file['gps_longitude'],
                         'long_velocity' : json_file['long_velocity'],
                         'lat_velocity ' : json_file['lat_velocity'],
                         'long_accel' : json_file['long_accel'],
                         'lat_accel' : json_file['lat_accel'],
                         'yaw' : json_file['yaw'],
                         'roll': json_file['roll'],
                         'pitch': json_file['pitch'],
                        'label_counts':{}}
    for instance in json_file['annotations']:
        if instance['type'] == 'bbox':
            label = instance['label']
            if (label == 'ptw') or  (label == 'vehicle'):
                att = '_'+instance['attribute']
                merged_meta['label_counts'][label+att] = merged_meta['label_counts'].get(label+att, 0) + 1
            else :
                merged_meta['label_counts'][label] = merged_meta['label_counts'].get(label, 0) + 1
    total_json.append(merged_meta)



    # 전체 파일 DF화
df = pd.DataFrame(total_json)
df1 = df.drop(['label_counts'], axis=1).groupby(['batch', 'filename']).first()
df2 = pd.concat([df[['batch','filename']], pd.DataFrame(df['label_counts'].tolist())], axis=1).groupby(['batch','filename']).sum()
group_df = pd.merge(df1, df2, on=['batch','filename'])
group_df.to_csv('80K.csv', index=True, encoding='utf-8')

[]


0it [00:00, ?it/s]

0it [00:00, ?it/s]

KeyError: "['label_counts'] not found in axis"